In [0]:
%python
spark.conf.set("fs.azure.account.key.cs0st0rage.dfs.core.windows.net", dbutils.secrets.get(scope = 'az_cs_scope', key= 'acc-key'))

In [0]:
CREATE OR REPLACE TABLE gold
AS SELECT * FROM DELTA.`abfss://gold@cs0st0rage.dfs.core.windows.net/` 

1. List the names of tables used.

In [0]:
SHOW TABLES;

2. Capture the SQL queries/ PySpark code snippets used for data cleansing.

3. Capture the SQL queries/ PySpark code snippets used for data transformations.

4. Create a master table named t_rating_desc as follows:

In [0]:
CREATE OR REPLACE TABLE t_rating_desc AS
SELECT *
FROM VALUES
    (0, 'No Rating'),
    (1, 'Poor'),
    (2, 'Average'),
    (3, 'Good'),
    (4, 'Excellent'),
    (5, 'Must Try') AS t(RATING, RATING_DESC);

5. Write a query that retrieves Restaurant Name, Rating and the corresponding Rating Description. Use the next whole number to get rating description. For the restaurants with blank rating, display the one corresponding to 0. 


In [0]:
SELECT Restaurant_Name, CEIL(g.Rating) AS Rating, Rating_desc FROM gold g,t_rating_desc t WhERE t.rating = g.rating;

6. Write a query to get Restaurant Name, City and Rating for Domino's Pizza so that the highest rated is displayed on top. If there are 2 outlets with the same rating, the first on the list should be based on the city in alphabetical order.

In [0]:
SELECT Restaurant_Name, City, Rating FROM gold WHERE Restaurant_Name = "Domino's Pizza" ORDER BY Rating DESC, City ASC

7. Find the number of restaurants with rating above 4 for cusine 'North Indian'.

In [0]:
SELECT Cuisine, count(Cuisine) AS Count FROM gold WHERE Cuisine = 'North Indian' and rating > 4 group by Cuisine

8. Find the number of restaurants with a rating below the 3.


In [0]:
SELECT count(*) AS Rest_Count FROM gold WHERE rating < 3

9. Find the number of people who have rated restaurants with cusine 'Fast Food,American'

In [0]:
SELECT concat(sum(Rating_count),'+ Ratings') as Rating_Count FROM gold where Cuisine = "Fast Food,American"

10. Find the restaurant in LuluMall Kochi that has the highest number of people rated.

In [0]:
SELECT Restaurant_Name, Address, Rating, Rating_count FROM gold WHERE lower(City) = 'lulu mall,kochi' ORDER BY Rating_Count DESC LIMIT 1

11. Identify the city with maximum number of restaurants.

In [0]:
SELECT City, count(Restaurant_id) as No_of_Restaurants FROM gold GROUP BY City ORDER BY No_of_Restaurants DESC LIMIT 1

12. Identify the number of Baskin Robbins outlets with a rating of 4.5 and above.

In [0]:
  SELECT Restaurant_Name, count(*) as Count FROM gold WHERE Restaurant_Name = 'Baskin Robbins' and Rating >= 4.5 GROUP BY Restaurant_Name

13. Identify the restaurants that sell least expensive Ice cream yet with a rating of 4 and above in Bangalore.

In [0]:
SELECT Restaurant_Name, City, Rating, m.Category, m.Item_name, m.price
FROM gold
LATERAL VIEW EXPLODE(Menu_Items) exploded AS m
WHERE Rating >= 4
  AND LOWER(City) like '%bangalore%'
  AND (
      LOWER(m.category) = 'ice cream'
      OR LOWER(m.category) = 'ice creams'
      OR LOWER(m.category) LIKE 'ice cream cup%'
  ) ORDER BY m.price;


14. Find the top 10 cities as per the number of restaurants listed on Swiggy.

In [0]:
SELECT City, count(Restaurant_id) as No_of_Restaurants FROM gold GROUP BY City ORDER BY No_of_Restaurants DESC LIMIT 10

15. Identify the cuisine that has the top rating in this dataset

In [0]:
SELECT Cuisine, Rating
FROM (
    SELECT Cuisine, 
           Rating,
           RANK() OVER (ORDER BY Rating DESC) AS ranks
    FROM gold
) AS t
WHERE ranks = 1;

16. What is the total number of restaurants listed in swiggy in the city Kannur with a rating count of 100+ ratings?

In [0]:
SELECT City, count(*) FROM gold WHERE City LIKE '%Kannur%' and Rating_count >= 100 GROUP BY City

17. Identify the restaurants that serve more than 1 cuisine. Display the Restaurant Name and the Cuisine

In [0]:
SELECT Restaurant_name, Cuisine FROM gold WHERE Cuisine LIKE '%,%'

18. Write a query to list the restaurant names with the letter ‘d’ in the name and also the position of the letter d.

In [0]:
SELECT Restaurant_Name, instr(lower(Restaurant_Name), 'd') AS position FROM gold WHERE lower(Restaurant_Name) LIKE '%d%'

19. Write a query to display the restaurants that have two or more ‘o’ in the name.

In [0]:
SELECT Restaurant_Name FROM gold WHERE regexp_count(lower(Restaurant_Name), 'o') >= 2

20. Write a query to show the restaurant name and cuisine as follows.
- AB FOODS POINT	Beverages | Pizzas
- Janta Sweet House	Sweets | Bakery
- theka coffee desi	Beverages
- Singh Hut	Fast Food | Indian


In [0]:
SELECT Restaurant_Name, replace(Cuisine, ',', ' | ' ) AS Cuisine FROM gold 

21. Write a query to display the restaurant_name and total cost of restaurants that start with S. The total cost should be based on each city. The list should be sorted based on total cost (least total cost should be on top)

In [0]:
WITH City_Cost AS (
  SELECT 
    City, SUM(Cost) AS Total_Cost
  FROM gold
  WHERE 
    lower(Restaurant_Name) LIKE 's%' GROUP BY City
)
SELECT 
  g.Restaurant_Name, g.City, c.Total_Cost
FROM gold g
JOIN 
  City_Cost c ON g.City = c.City
WHERE lower(g.Restaurant_Name) LIKE 's%' 
ORDER BY c.Total_Cost;


22. Create a new table with the structure:

RESTAURANT_NAME	STRING,
LOCALITY	STRING,
CITY	STRING,
RATING	DOUBLE,
RATING_COUNT	STRING,

Load the data as follows: 
Pull the records with city as Electronic City,Bangalore with a rating above 4 and rating count of at least 50+
Split the value into Locality and City using code (Do not hardcode the values).

In [0]:
CREATE TABLE IF NOT EXISTS filtered_electronic_city (
    Restaurant_Name STRING,
    Locality STRING,
    City STRING,
    Rating DOUBLE,
    Rating_Count STRING
);

In [0]:
INSERT INTO filtered_electronic_city
SELECT 
    Restaurant_Name,
    split(City, ',')[0] AS Locality, -- Extract Locality
    split(City, ',')[1] AS City,     -- Extract City
    Rating,
    cast(Rating_Count as STRING)
FROM 
    gold
WHERE 
    lower(City) = 'electronic city,bangalore'
    AND Rating > 4
    AND Rating_Count >= 50;

23. Write a query to list the restaurant and city that serve a single cuisine.

In [0]:
SELECT Restaurant_Name, City, Cuisine
FROM gold
WHERE Cuisine NOT LIKE '%,%'

24. Identify restaurants with anything other than alphabets in the restaurant name. Display distinct values in the output.

In [0]:
SELECT DISTINCT Restaurant_Name
FROM gold
WHERE Restaurant_Name REGEXP '[^A-Za-z ]';

25. Analyze the entire dataset and detail one interesting finding.

In [0]:
SELECT 
    CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))  -- Get second part after comma
        ELSE City  -- If no comma, return the original city name
    END AS City,
    COUNT(Restaurant_Name) AS num_restaurants
FROM gold
WHERE Rating < 2.5
GROUP BY 
    CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))  -- Extract second part
        ELSE City  -- Return original city if no comma
    END
ORDER BY num_restaurants DESC
LIMIT 10;



In [0]:
SELECT CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))  -- Get second part after comma
        ELSE City  -- If no comma, return the original city name
    END AS City, 
    COUNT(Restaurant_Name) AS num_restaurants
FROM gold
GROUP BY 
    CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))  -- Extract second part
        ELSE City  -- Return original city if no comma
    END
ORDER BY num_restaurants DESC
LIMIT 10;


# Visualization

In [0]:
SELECT count(Restaurant_Id) FROM gold;

In [0]:
SELECT count(DISTINCT City) AS city_count FROM gold;

In [0]:
SELECT COUNT(DISTINCT m.Item_name) AS item_count
FROM gold
LATERAL VIEW EXPLODE(Menu_Items) AS m;

In [0]:
SELECT COUNT(DISTINCT c.cuisine) AS cuisine_count
FROM (
    SELECT EXPLODE(SPLIT(Cuisine, ',')) AS cuisine
    FROM gold
) AS c;


In [0]:
SELECT CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))
        ELSE City
    END AS City, COUNT(*) AS high_rated_restaurant_count
FROM gold
WHERE Rating >= 4.5
GROUP BY CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))
        ELSE City
    END
ORDER BY high_rated_restaurant_count DESC
LIMIT 3

In [0]:
WITH city_counts AS (
    SELECT 
        CASE 
            WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))
            ELSE City
        END AS City,
        COUNT(*) AS restaurant_count
    FROM gold
    GROUP BY CASE 
                WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))
                ELSE City
             END
),
top_3_cities AS (
    SELECT City, restaurant_count
    FROM city_counts
    ORDER BY restaurant_count DESC
    LIMIT 5
),
others_count AS (
    SELECT SUM(restaurant_count) AS other_restaurants_count
    FROM city_counts
    WHERE City NOT IN (SELECT City FROM top_3_cities)
)

SELECT 
    t.City,
    t.restaurant_count
FROM top_3_cities t
UNION ALL
SELECT 
    'Others' AS City,
    o.other_restaurants_count AS restaurant_count
FROM others_count o;


In [0]:
WITH Cuisines AS (
    SELECT *
    FROM (
    SELECT EXPLODE(SPLIT(Cuisine, ',')) AS cuisine, Restaurant_ID
    FROM gold
) AS c
)
SELECT cuisine, COUNT(cuisine) FROM Cuisines GROUP BY cuisine ORDER BY COUNT(cuisine) DESC LIMIT 5




In [0]:
SELECT 
    CASE 
        WHEN LOWER(m.Veg_or_Non_veg) == 'veg' THEN 'Veg'
        ELSE 'Non-Veg'
    END AS Dish_Type,
    COUNT(*) AS Dish_Count
FROM gold
LATERAL VIEW EXPLODE(Menu_Items) AS m
GROUP BY 
    CASE 
        WHEN
        LOWER(m.Veg_or_Non_veg) == 'veg' THEN 'Veg'
        ELSE 'Non-Veg'
    END;


In [0]:
SELECT Cost, Rating FROM gold WHERE Rating>0

In [0]:
SELECT 
    Restaurant_Name AS Franchise,
    COUNT(*) AS Restaurant_Count
FROM gold
GROUP BY Restaurant_Name
ORDER BY Restaurant_Count DESC
LIMIT 3


In [0]:
SELECT 
    CEIL(Rating) AS Rounded_Rating,  -- Round up the rating
    City, 
    COUNT(Restaurant_Name) AS num_restaurants
FROM (
    SELECT 
        CASE 
            WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))  -- Get second part after comma
            ELSE City  -- If no comma, return the original city name
        END AS City, 
        Restaurant_Name,
        Rating
    FROM gold
) AS CityRatingData
WHERE Rating > 0  -- Filter for positive ratings
GROUP BY City, CEIL(Rating)  -- Group by city and rounded rating
ORDER BY num_restaurants DESC
LIMIT 10;


In [0]:
-- Create a table to store metro cities
CREATE OR REPLACE TABLE metro_cities (
    city STRING
);

-- Insert hardcoded values for metro cities in India
INSERT INTO metro_cities (city)
VALUES 
    ('Mumbai'),
    ('Delhi'),
    ('Bangalore'),
    ('Chennai'),
    ('Kolkata'),
    ('Hyderabad'),
    ('Pune'),
    ('Ahmedabad');


In [0]:
SELECT 
    exploded_cuisine.Cuisine,
    CASE 
        WHEN City LIKE '%,%' THEN TRIM(SUBSTRING(City, CHARINDEX(',', City) + 1, LEN(City)))
        ELSE City
    END AS main_city,
    COUNT(*) AS restaurant_count
FROM gold
LATERAL VIEW EXPLODE(SPLIT(Cuisine, ',')) exploded_cuisine AS Cuisine
GROUP BY 
    main_city, 
    exploded_cuisine.Cuisine
HAVING LOWER(main_city) IN (SELECT LOWER(city) FROM metro_cities) AND lower(Cuisine) IN ("north indian",
    "south indian",
    "chinese",
    "mughlai",
    "biryani",
    "continental",
    "italian",
    "street food",
    "punjabi",
    "desserts");


In [0]:
SELECT 
    avg(cost),m.veg_or_non_veg AS Food_Type
FROM gold
LATERAL VIEW EXPLODE(Menu_Items) AS m
GROUP BY 
    m.Veg_or_Non_veg

In [0]:
SELECT count(m.item_name) AS Counts, m.item_name from gold LATERAL VIEW explode(Menu_Items) AS m GROUP BY m.item_name ORDER BY Counts DESC LIMIT 100 ;